<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_D3_MongoDB_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

##References
https://stackoverflow.com/questions/45142629/cant-connect-to-mongo-db-via-spark <br>
https://docs.mongodb.com/spark-connector/current/python-api/

#Packages

In [1]:
from pymongo import MongoClient
import pandas as pd
import os

#Install & Test LOCAL MongoDB

In [2]:
!apt install mongodb > /dev/null
!service mongodb start



 * Starting database mongodb
   ...done.


In [3]:
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']

['admin', 'local']

In [4]:
db = client.local
#db = client.sparkDB
#db = client.people
db.list_collection_names()

['startup_log']

In [5]:
table = db.startup_log
#table = db.coll01
#table = db.contacts
#table.count_documents({}) #gives the number of documents in the table
table.find_one()

{'_id': '3d1d6961c85d-1665133716464',
 'hostname': '3d1d6961c85d',
 'startTime': datetime.datetime(2022, 10, 7, 9, 8, 36),
 'startTimeLocal': 'Fri Oct  7 09:08:36.464',
 'cmdLine': {'config': '/etc/mongodb.conf',
  'net': {'bindIp': '127.0.0.1'},
  'storage': {'dbPath': '/var/lib/mongodb', 'journal': {'enabled': True}},
  'systemLog': {'destination': 'file',
   'logAppend': True,
   'path': '/var/log/mongodb/mongodb.log'}},
 'pid': 471,
 'buildinfo': {'version': '3.6.3',
  'gitVersion': '9586e557d54ef70f9ca4b43c26892cd55257e1a5',
  'modules': [],
  'allocator': 'tcmalloc',
  'javascriptEngine': 'mozjs',
  'sysInfo': 'deprecated',
  'versionArray': [3, 6, 3, 0],
  'openssl': {'running': 'OpenSSL 1.1.1  11 Sep 2018',
   'compiled': 'OpenSSL 1.1.1  11 Sep 2018'},
  'buildEnvironment': {'distmod': '',
   'distarch': 'x86_64',
   'cc': 'cc: cc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
   'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pr

#Connect to and Test Remote MongoDB

#Install Spark

## Deprecated - dont use these commands

In [ ]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#

#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark              --- no more required,  replaced by pyspark
!pip install -q pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 55.3 MB/s 


## Simplified Install 2022

In [6]:
!pip3 install -q pyspark

     |████████████████████████████████| 281.3 MB 50 kB/s 
     |████████████████████████████████| 199 kB 63.6 MB/s 


#Spark2Mongo

In [7]:
#import findspark
#findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/sparkDB.coll01") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/sparkDB.coll01") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
    .getOrCreate()

In [ ]:
#sc.stop()

In [8]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

##Write to MongoDB

In [9]:
people = spark.createDataFrame([("Bilbo Baggins",  50), ("Gandalf", 1000), ("Thorin", 195), ("Balin", 178), ("Kili", 77),
   ("Dwalin", 169), ("Oin", 167), ("Gloin", 158), ("Fili", 82), ("Bombur", None)], ["name", "age"])

In [10]:
people.show()

+-------------+----+
|         name| age|
+-------------+----+
|Bilbo Baggins|  50|
|      Gandalf|1000|
|       Thorin| 195|
|        Balin| 178|
|         Kili|  77|
|       Dwalin| 169|
|          Oin| 167|
|        Gloin| 158|
|         Fili|  82|
|       Bombur|null|
+-------------+----+



In [11]:
# this will send the data to the default database / collection specified during startup
people.write.format("mongo").mode("append").save()

In [12]:
# this will send the data to a new database / collection specified in this command
people.write.format("mongo").mode("append").option("database","people").option("collection", "contacts").save()

In [13]:
client.list_database_names() # ['admin', 'local']

['admin', 'local', 'people', 'sparkDB']

##Check with MongoDB client

In [14]:
client.list_database_names() # ['admin', 'local']

['admin', 'local', 'people', 'sparkDB']

In [15]:
db = client.people
list(db.list_collections())

[{'name': 'contacts',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b'\xf7L\xc8\xec\x92~E\x1d\xael-\x14\xf2\x15\xd1\xbb', 4)},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'people.contacts'}}]

In [16]:
db2 = client.sparkDB
list(db2.list_collections())

[{'name': 'coll01',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': Binary(b'{n\xd5\xc0O\x89D+\xb8\x96\xd2\xd6Q\xea\xc4\xc6', 4)},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'sparkDB.coll01'}}]

In [17]:
kollection = db.contacts
c = kollection.find({},{"_id":0})
for crnt in c:
    print(crnt)

{'name': 'Bilbo Baggins', 'age': 50}
{'name': 'Gandalf', 'age': 1000}
{'name': 'Thorin', 'age': 195}
{'name': 'Balin', 'age': 178}
{'name': 'Kili', 'age': 77}
{'name': 'Dwalin', 'age': 169}
{'name': 'Oin', 'age': 167}
{'name': 'Gloin', 'age': 158}
{'name': 'Fili', 'age': 82}
{'name': 'Bombur'}


In [18]:
pandasDF = pd.DataFrame(kollection.find({},{"_id":0}))
pandasDF

,name,age
0,Bilbo Baggins,50.0
1,Gandalf,1000.0
2,Thorin,195.0
3,Balin,178.0
4,Kili,77.0
5,Dwalin,169.0
6,Oin,167.0
7,Gloin,158.0
8,Fili,82.0
9,Bombur,NaN


##Read from MongoDB

In [19]:
sparkDF = spark.read.format("mongo").option("uri","mongodb://127.0.0.1/people.contacts").load()
sparkDF.show()

+--------------------+----+-------------+
|                 _id| age|         name|
+--------------------+----+-------------+
|{633fed0d4b8e5264...|  50|Bilbo Baggins|
|{633fed0d4b8e5264...|1000|      Gandalf|
|{633fed0d4b8e5264...| 195|       Thorin|
|{633fed0d4b8e5264...| 178|        Balin|
|{633fed0d4b8e5264...|  77|         Kili|
|{633fed0d4b8e5264...| 169|       Dwalin|
|{633fed0d4b8e5264...| 167|          Oin|
|{633fed0d4b8e5264...| 158|        Gloin|
|{633fed0d4b8e5264...|  82|         Fili|
|{633fed0d4b8e5264...|null|       Bombur|
+--------------------+----+-------------+



In [20]:
from datetime import datetime
import pytz
print('signed off at  ',datetime.now(pytz.timezone('Asia/Kolkata')))

signed off at   2022-10-07 14:41:34.278901+05:30


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)